In [1]:
cd ..

/mnt/disk2/samarth/qdpy-numpyro


In [2]:
cd qdpy_jax

/mnt/disk2/samarth/qdpy-numpyro/qdpy_jax


In [3]:
ls

1walk1iter.py             load_multiplets.py           rotation_kernels.py
build_cenmult_and_nbs.py  muhz.dat                     test_jax_class.py
build_supermatrix.py      namedtuple_Cvec.py           U3672.dat
class_Cvec.py             nl.dat                       V3672.dat
globalvars.py             one_walker_one_iteration.py  w.dat
gnool_jit.py              prune_multiplets.py          wigner_map2.py
__init__.py               __pycache__/                 wigner_map.py
jax_functions.py          pyrotry.py
jax_rotation_kernels.py   r.dat


In [4]:
from IPython.display import set_matplotlib_formats
from jax.scipy.special import logsumexp
import matplotlib.pyplot as plt
from jax import random, vmap
import jax.numpy as jnp
import seaborn as sns
import pandas as pd
import numpy as np
import os

NAX = np.newaxis

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro import handlers

plt.style.use('bmh')
if "NUMPYRO_SPHINXBUILD" in os.environ:
    set_matplotlib_formats('svg')

assert numpyro.__version__.startswith('0.6.0')
#numpyro.set_platform("gpu")

numpyro.set_platform("cpu")
#numpyro.set_host_device_count(5)

from scipy.integrate import simps
from jax.numpy import trapz